<a href="https://colab.research.google.com/github/thales20266/Projeto-Transfer-Learning/blob/main/Assistente_Virtual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install SpeechRecognition==3.8.1 pydub wikipedia gTTS playsound==1.2.2
!apt-get install -y ffmpeg libsndfile1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.7 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=4a7d6b3fda58b25d0cf9f6cf45432f26d97ce69cc565105a73da5fec7061c987
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [7]:
import os
import io
from IPython.display import Audio, display, HTML
from pydub import AudioSegment

def convert_to_wav(input_path: str, output_path: str = None, sample_rate: int = 16000):
    """Converte arquivo de áudio para WAV PCM 16k mono (compatível com bibliotecas STT)."""
    if output_path is None:
        base, _ = os.path.splitext(input_path)
        output_path = f"{base}_conv.wav"
    audio = AudioSegment.from_file(input_path)
    audio = audio.set_frame_rate(sample_rate).set_channels(1)
    audio.export(output_path, format='wav')
    return output_path

wav_path = convert_to_wav("/content/novobalanço.mp3")
print("Arquivo convertido:", wav_path)



Arquivo convertido: /content/novobalanço_conv.wav


In [12]:
# 3) Speech-to-Text: usando SpeechRecognition + Google Web Speech API
# =====================
import speech_recognition as sr




def speech_to_text_google(wav_path: str) -> str:

#Transcreve arquivo WAV usando o recognizer do Google (SpeechRecognition).
#Observação: Requer internet; é gratuito para pequenos usos, mas sujeito a limites.

  r = sr.Recognizer()
  with sr.AudioFile(wav_path) as source:
    audio = r.record(source)
  try:
    text = r.recognize_google(audio, language='pt-BR')
    return text
  except sr.UnknownValueError:
    return "[Não foi possível entender o áudio]"
  except sr.RequestError as e:
    return f"[Erro na requisição ao serviço de reconhecimento: {e}]"

In [13]:
# 4) Text-to-Speech: usando gTTS
# =====================
from gtts import gTTS




def text_to_speech(text: str, lang: str = 'pt', out_path: str = 'tts_output.mp3') -> str:
#Gera arquivo MP3 com a fala do texto e retorna o caminho do arquivo."""
  tts = gTTS(text=text, lang=lang)
  tts.save(out_path)
  return out_path

In [14]:
# 5) Módulo de interpretação de comandos (NLU simples - intent rules)
# =====================
import re
import urllib.parse
import wikipedia


wikipedia.set_lang('pt')




def interpretar_comando(texto: str) -> dict:
#Analisa a frase e retorna um dicionário com intent e parâmetros.
#Intent simples baseada em palavras-chave.
#Exemplos de intents: wikipedia_search, open_youtube, find_pharmacy, say_time, unknown

  t = texto.lower()


# wikipedia
  if 'wikipedia' in t or 'wikipédia' in t or 'pesquise' in t and 'wikipedia' in t:
  # tenta extrair o tópico
    match = re.search(r'(?:(pesquise|procure|buscar) por|sobre) (.+)', t)
    topic = match.group(2) if match else t
    return {'intent': 'wikipedia_search', 'topic': topic}


# youtube
  if 'youtube' in t or 'vídeo' in t or 'toca' in t and 'youtube' in t:
    match = re.search(r'(?:youtube|vídeo) (?:sobre|de|sobre o|sobre a)?\s*(.*)', t)
    topic = match.group(1) if match and match.group(1).strip() else ''
    return {'intent': 'open_youtube', 'topic': topic}


# find pharmacy / farmácia
  if 'farmácia' in t or 'farmacia' in t or 'remédio' in t:
    return {'intent': 'find_pharmacy'}


# hora
  if 'hora' in t or 'que horas' in t:
    return {'intent': 'say_time'}


# saudação
  if any(x in t for x in ['olá', 'oi', 'bom dia', 'boa tarde', 'boa noite']):
    return {'intent': 'greeting'}


# comandos de abrir site simples
  match_open = re.search(r'abrir (.+)', t)
  if match_open:
    site = match_open.group(1)
    return {'intent': 'open_website', 'site': site}


  return {'intent': 'unknown'}

In [15]:
# 6) Executor de intents (ações automatizadas)
# =====================
import datetime
from IPython.display import HTML




def executar_intent(intent: dict) -> str:
#Executa a ação correspondente à intent e retorna uma resposta textual."""
  it = intent.get('intent')
  if it == 'wikipedia_search':
    topic = intent.get('topic', '')
    try:
      summary = wikipedia.summary(topic, sentences=2)
      return f"WIKIPEDIA: {summary}"
    except Exception as e:
      return f"Não encontrei informação sobre '{topic}' na Wikipedia. ({e})"

  if it == 'open_youtube':
    topic = intent.get('topic', '')
    query = urllib.parse.quote(topic or '')
    url = f'https://www.youtube.com/results?search_query={query}'
    return f'Abra este link no navegador para ver resultados no YouTube: {url}'

  if it == 'find_pharmacy':
    url = 'https://www.google.com/maps/search/pharmacy+near+me'
    return f'Abra este link para encontrar farmácias próximas: {url}'

  if it == 'say_time':
    now = datetime.datetime.now()
    return f'Agora são {now.hour:02d}:{now.minute:02d}.'

  if it == 'greeting':
    return 'Olá! Como posso ajudar você hoje?'

  if it == 'open_website':
    site = intent.get('site')
  # tenta criar url plausível
    if not site.startswith('http'):
      if ' ' in site:
        site_query = urllib.parse.quote(site)
        url = f'https://www.google.com/search?q={site_query}'
      else:
        url = f'https://{site}'
    else:
      url = site
    return f'Abra este link: {url}'

  return "Desculpe, não entendi o comando."

In [16]:
# 7) Pipeline principal: gravar/transcrever/interpretar/executar
# =====================


def pipeline_from_file(wav_path: str, speak_response: bool = True) -> dict:
#Processa um arquivo WAV: transcreve, interpreta e executa a intent.
#Retorna dicionário com keys: transcript, intent, action_response, tts_path (opcional)

  transcript = speech_to_text_google(wav_path)
  intent = interpretar_comando(transcript)
  action_response = executar_intent(intent)

  tts_path = None
  if speak_response:
    tts_path = text_to_speech(action_response, lang='pt', out_path='assistant_reply.mp3')
  return {
    'transcript': transcript,
    'intent': intent,
    'action_response': action_response,
    'tts_path': tts_path
}